<a href="https://colab.research.google.com/github/YinterestingProjects/human-wildlife-interactions/blob/main/notebooks/cluster_translation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install pandas==1.4.2
! pip install fasttext
! pip install transformers
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 55.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4391756 sha256=85d1f72d462c782abc528f51bb88ffcf75095643dc4dde68f20cb5fff088dd06
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/publi

In [3]:
import pickle
import pandas as pd
import numpy as np
pd.__version__

'1.4.2'

In [4]:
# directory setup
user = "sally"
#user = "jina"
#user = "josh"
     
try:
  if user == "sally":
    directory = '/content/drive/MyDrive/MADS/MADS_Capstone'
  elif user == "jina":
    directory = "JINA: INSERT YOUR PATH HERE"
  elif user == "josh":
    directory = "/content/drive/MyDrive/MADS_Capstone"
except:
  dprint("No user has been selected")

In [5]:
vDets_fp = f'{directory}/data/processed/videoDets.pkl'
vComs_fp = f'{directory}/data/processed/videoComs.pkl'

with open(vDets_fp, 'rb') as f:
  vDets_df = pickle.load(f)

In [6]:
vDets_df.head(2)
# vDets_df.info()

,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,localizations.zh-Hans.title,localizations.zh-Hans.description,localizations.pt.title,localizations.pt.description,localizations.zu.title,localizations.zu.description,localizations.af.title,localizations.af.description,localizations.pt-PT.title,localizations.pt-PT.description
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# check if the localization worked
vDets_df['title_translated'] = np.where(vDets_df['snippet.title'] == vDets_df['snippet.localized.title'], True, False)
vDets_df['descrip_translated'] = np.where(vDets_df['snippet.description'] == vDets_df['snippet.localized.description'], True, False)

vDets_df[vDets_df['title_translated']==False] #0
vDets_df[vDets_df['descrip_translated']==False] #0

# check on field LiveBroadcastContent
vDets_df['snippet.liveBroadcastContent'].value_counts()	# all none

vDets_df['contentDetails.caption'].value_counts() # false 3818, true 77

false    3818
true       77
Name: contentDetails.caption, dtype: int64

In [8]:
# identify columns to use for clustering
cols_str = '''yt8M_id, id, snippet.title, snippet.description, snippet.tags'''
cols_to_keep = cols_str.split(', ')
cols_to_keep

['yt8M_id', 'id', 'snippet.title', 'snippet.description', 'snippet.tags']

In [9]:
# sample data for testing
title_des_df = vDets_df.loc[:, cols_to_keep]
sample_df = title_des_df.sample(20, random_state=42)     # 1985 english + japanese
sample_df.head()

,yt8M_id,id,snippet.title,snippet.description,snippet.tags
3682,1dhU,TedONA50f8Y,富山トラウト釣行 黄金イワナ Trout fishing trip in Toyama,2011年5月2日、釣り仲間3人と富山の河川へトラウト釣行に。\r\n黄金色のイワナを釣るこ...,"[釣り, フィッシング, 川, 本流, 渓流, トラウト, イワナ, ヤマメ, UFJ釣行,..."
912,SyhK,MydM_86g5-8,Fiordland NZ: Mount Titiroa,The Lure of Manapouri's Mt Titiroa where a gro...,"[Spiritual, Movie, Tourism, Spirituality, Moun..."
321,1dNW,q9c_a805j5M,A Moment With Serena,She's pretty entertaining one-on-one. \n\n-- F...,"[tiger, big cat, serena, snuffle, cute, Cat, b..."
2098,hUtY,xnsYaPjI0pw,Презентация-Дикие и домашние животные.,"Школьные презентации ""ОКРУЖАЮЩИЙ МИР""-""Дикие и...","[Презентации, слайд-шоу, окружающий мир, видео..."
1480,dOax,A_qDbbb1tsI,Visite du zoo de Maubeuge 07/07/2012,,"[Zoo, Maubeuge, animaux, jardin, zoologique, e..."


In [10]:
import fasttext
from transformers import pipeline
#from multilingual_translation import translation as translate
#import sentencepiece


# Load the pre-trained fasttext language detection model, translation pipeline with pre-trained model
language_detector = fasttext.load_model(f'{directory}/models/lid.176.ftz')
#translator = pipeline('translation', model='Helsinki-NLP/opus-mt-fr-en', tokenizer='Helsinki-NLP/opus-mt-fr-en')
translator = pipeline('translation', model='facebook/m2m100_418M')


In [11]:
from tqdm import tqdm
import re


def clean_text(text):
  # remove spacing
  text = text.replace('\r\n', '')
  text = text.replace('\n', '')

  # remove urls
  text = re.sub(r'http\S+', '', text)

  return text
  
  
def detect_language(text):
    '''function to detect the language of a single text string'''
    labels, scores = language_detector.predict(text)
    label = labels[0].replace("__label__", '')
    score = round(scores[0], 4)

    return label, score

def translate_text(text, source_language, target_language):
    '''function to translate a single text string from a foreign language to English'''
    if source_language == 'en':
      return text
    else:
      try: 
        results = translator(text, src_lang=source_language, tgt_lang=target_language, max_length=500)
        return results[0]['translation_text']
      except KeyError:
        return 'source language unavailable for translation'

def translate_df_column(df, column_to_translate, col_prefix):
    '''function to translate a specified column in a Pandas DataFrame from a foreign language to English'''
    
    prefix = col_prefix
    # cleaning
    df[f'{prefix}_clean'] = df[column_to_translate].apply(clean_text)
    
    # language detection
    df[f'{prefix}_src_lang'], df[f'{prefix}_src_conf'] = zip(*df[f'{prefix}_clean'].apply(detect_language))

    # english translation
    tqdm.pandas(desc=f'{prefix} translation')
    df[f'{prefix}_en'] = df.progress_apply(lambda x: translate_text(x[f'{prefix}_clean'], 
                                                                    x[f'{prefix}_src_lang'], 
                                                                    'en'
                                                                    ), axis=1)
    
    return df

In [12]:
#translated_df = translate_df_column(title_des_df, 'snippet.title', 'title')
#translated_df.to_pickle(f'{directory}/data/processed/translated.pkl')
with open(f'{directory}/data/processed/translated.pkl', 'rb') as translated_file:
  translated_df = pickle.load(translated_file)
translated_df.head()

,yt8M_id,id,snippet.title,snippet.description,snippet.tags,title_clean,title_src_lang,title_src_conf,title_en
0,HTft,130S1TN3P0M,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,"[Łoś, Alces alces, Elk (Animal), wiewiórka, Sq...",Obserwator 22: Jesień na tamie / Autumn on a ...,pl,0.3335,Observer 22: Autumn on a Beaver's Dam
1,VX3C,iHpgw8WN6Bg,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...","[zoo, isère, le, domaine, des, fauves, ocelots...",bébés ocelots et otocyons au Domaine des Fauves,fr,0.9698,Babies and Otocyons in the Domaine des Fauves
2,j2Er,XrX9NBMUOzw,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,"[wildlife, giraffe, sfzoobehindthescenes, birt...",Two Giraffe Calves born at the San Francisco Zoo,en,0.9882,Two Giraffe Calves born at the San Francisco Zoo
3,3HOO,qwL5HKti-dw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,"[Cerbi carpatini, Capriori, Deer, Landscapes.,...",Fauna salbatica bucovineana in fotografi - Buk...,en,0.2253,Fauna salbatica bucovineana in fotografi - Buk...
4,6nfW,wPJy9D_kzVU,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,"[New, Project]",Wyoming Archery Mule Deer Hunt With Joe Thomas...,en,0.6670,Wyoming Archery Mule Deer Hunt With Joe Thomas...


In [13]:
desc_translated_df = translate_df_column(translated_df, 'snippet.description', 'desc')
desc_translated_df.to_pickle(f'{directory}/data/processed/desc_translated_test.pkl')
desc_translated_df.head()

desc translation:   1%|          | 36/3895 [07:24<13:14:01, 12.35s/it]


KeyboardInterrupt: ignored

### sandbox

In [ ]:
translated_df.groupby(['title_src_lang'])['id'].count().sort_values(ascending=False)#.plot(kind='bar')

title_src_lang
en    2557
es     208
de     177
fr     167
ja      91
      ... 
az       1
lb       1
lv       1
mr       1
my       1
Name: id, Length: 61, dtype: int64

In [ ]:
translate_df_column(sample_df, 'snippet.title', 'title')

translation: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


,yt8M_id,id,snippet.title,snippet.description,snippet.tags,desc_src_lang,desc_src_conf,title_src_lang,title_src_conf,source_lang,source_lang_conf,title_clean,title_en
3682,1dhU,TedONA50f8Y,富山トラウト釣行 黄金イワナ Trout fishing trip in Toyama,2011年5月2日、釣り仲間3人と富山の河川へトラウト釣行に。\r\n黄金色のイワナを釣るこ...,"[釣り, フィッシング, 川, 本流, 渓流, トラウト, イワナ, ヤマメ, UFJ釣行,...",ja,0.8059,ja,0.5948,ja,0.5948,富山トラウト釣行 黄金イワナ Trout fishing trip in Toyama,Trout fishing trip in Toyama
912,SyhK,MydM_86g5-8,Fiordland NZ: Mount Titiroa,The Lure of Manapouri's Mt Titiroa where a gro...,"[Spiritual, Movie, Tourism, Spirituality, Moun...",en,0.9561,en,0.3247,en,0.3247,Fiordland NZ: Mount Titiroa,Fiordland NZ: Mount Titiroa
321,1dNW,q9c_a805j5M,A Moment With Serena,She's pretty entertaining one-on-one. \n\n-- F...,"[tiger, big cat, serena, snuffle, cute, Cat, b...",en,0.3994,en,0.2542,en,0.2542,A Moment With Serena,A Moment With Serena
2098,hUtY,xnsYaPjI0pw,Презентация-Дикие и домашние животные.,"Школьные презентации ""ОКРУЖАЮЩИЙ МИР""-""Дикие и...","[Презентации, слайд-шоу, окружающий мир, видео...",ru,0.9754,ru,0.9746,ru,0.9746,Презентация-Дикие и домашние животные.,Wild and domestic animals.
1480,dOax,A_qDbbb1tsI,Visite du zoo de Maubeuge 07/07/2012,,"[Zoo, Maubeuge, animaux, jardin, zoologique, e...",en,0.1245,fr,0.6776,fr,0.6776,Visite du zoo de Maubeuge 07/07/2012,Visit to the Zoo of Maubeuge 07/07/2012
366,CBnQ,nj0t3zi2rKY,Wild Animals,This video was shot at Vision Quest Wild Anima...,"[Wild, Animals, Vision, Quest, Ranch, Bed, Bre...",en,0.9425,de,0.3787,de,0.3787,Wild Animals,Wild Animals
1894,NfCl,xGABQMb17nE,Country Bear and Cute Baby Cubs | BBC Studios,Want more natural history and wildlife videos?...,"[country, bears, polar, cute, baby, animals, w...",en,0.8596,en,0.6520,en,0.6520,Country Bear and Cute Baby Cubs | BBC Studios,Country Bear and Cute Baby Cubs | BBC Studios
1503,HIee,GCB6KGQcmyo,KENYA 2011 BOOM,MASAI MARA TO BOUNCE BACK FROM CRISES...GREEN ...,"[KENYA, 2011, masai, mara, safari, african]",vi,0.6909,en,0.1245,en,0.1245,KENYA 2011 BOOM,KENYA 2011 BOOM
2623,9sFJ,FR5paiKmWYY,A Visit to Sydney Aquarium & Animal World,I had the opportunity to walk around these fas...,"[Sydney, animals, sealife, wildlife, Australia...",en,0.9907,en,0.4160,en,0.4160,A Visit to Sydney Aquarium & Animal World,A Visit to Sydney Aquarium & Animal World
2992,ejaE,vV7kM-h7hsc,Cazadores por el Mundo.Territorios Salvajes,El programa de Cazavisión Cazadores por el Mun...,"[rececho, gamos, venados, cazavisión, grupo v]",es,0.9395,es,0.9710,es,0.9710,Cazadores por el Mundo.Territorios Salvajes,Hunting for the World: Wild Territories


In [ ]:
sample_df.to_pickle(f'{directory}/data/processed/sample.pkl')

In [ ]:
sample_df['snippet.title']#.apply(detect_language)

3682          富山トラウト釣行　黄金イワナ Trout fishing trip in Toyama
912                           Fiordland NZ: Mount Titiroa
321                                  A Moment With Serena
2098               Презентация-Дикие и домашние животные.
1480                 Visite du zoo de Maubeuge 07/07/2012
366                                          Wild Animals
1894        Country Bear and Cute Baby Cubs | BBC Studios
1503                                      KENYA 2011 BOOM
2623            A Visit to Sydney Aquarium & Animal World
2992          Cazadores por el Mundo.Territorios Salvajes
2392                                         Kenia Safari
1657                                      pups at 5 weeks
3882    B3 T2 L6 Mating Systems & Parental Care (Audio...
2855                  Okavango Delta Boat Safari Botswana
2827    Stealth Gear One Man Photography Chair Hide Re...
3639    Yorkshire Wildlife Park and  Painted Dog Conse...
1602        Fauna Flash: Konik paarden in de uiterwaarden
14      CABALL

In [ ]:
detect_language(sample_df['snippet.title'][3682])

('ja', 0.5948)

In [ ]:
sample_df['desc_src_lang'], sample_df['desc_src_conf']= zip(*sample_df['snippet.description'].apply(detect_language))
sample_df['title_src_lang'], sample_df['title_src_conf']= zip(*sample_df['snippet.title'].apply(detect_language))
sample_df

ValueError: ignored

In [ ]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 3682 to 1431
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   yt8M_id              20 non-null     object 
 1   id                   20 non-null     object 
 2   snippet.title        20 non-null     object 
 3   snippet.description  20 non-null     object 
 4   snippet.tags         18 non-null     object 
 5   source_lang          20 non-null     object 
 6   source_conf          20 non-null     float64
dtypes: float64(1), object(6)
memory usage: 1.2+ KB


In [ ]:
sample_df['snippet.description'][3682]

'2011年5月2日、釣り仲間3人と富山の河川へトラウト釣行に。\r\n黄金色のイワナを釣ることが出来ました。\r\n\r\nBegining of May 2011,\r\nAt Toyama pref. in Japan\r\nThere`s still some snow at mountain stream area,\r\nI fished golden char by chance,which is not usual.\r\n\r\nBlog:UFJ釣行 /UFJ Fishing\r\n http://ufj.naturum.ne.jp/\r\n\r\n Twitter: \r\nhttp://twitter.com/#!/Bo_Rovan'

In [ ]:
def clean_text(text):
  # remove spacing
  text = text.replace('\r\n', '')

  # remove urls
  text = re.sub(r'http\S+', '', text)

  return text


In [ ]:
translator(clean_text(sample_df['snippet.description'][3682]), src_lang='ja', tgt_lang ='en')

[{'translation_text': 'Begining of May 2011, At Toyama pref. in JapanThere`s still some snow at mountain stream area, I fished golden char by chance,which is not usual.Blog:UFJ Fishing /UFJ Fishing Twitter:'}]

In [ ]:
translator(clean_text(sample_df['snippet.description'][2098]), src_lang='ru', tgt_lang ='en')

[{'translation_text': 'School presentations "Around the World"-"Wild and domestic animals." watch online movies on'}]

In [ ]:
sample_df['snippet.description'][2098]

'Школьные презентации "ОКРУЖАЮЩИЙ МИР"-"Дикие и домашние животные." http://lerusik.taba.ru/\nсмотреть онлайн фильмы на http://kinotron.ucoz.ru/'

In [ ]:
# sample a group to test translator
test_df = translated_df.sample(20, random_state=21)
test_df.groupby(['desc_src_lang']).size()

desc_src_lang
de    1
en    8
es    3
fr    3
it    1
ja    1
pt    1
sh    1
th    1
dtype: int64

In [ ]:
buffer = []
for lang, group in test_df.groupby(['desc_src_lang']):
  print(lang)
  print(len(group))
  #print(group['snippet.description'].to_list())
  if lang == 'en':
    buffer.append('en')
  else:
    try:
      results = translator(group['snippet.description'].to_list(), batch_size = 10, src_lang=lang, tgt_lang='en', max_length=400)
      buffer.append(results)
    except KeyError:
      buffer.append('translation not available for this language')

de
1
en
8
es
3
fr
3


Your input_length: 1253 is bigger than 0.9 * max_length: 400. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


it
1
ja
1
pt
1
sh
1
th
1


In [ ]:
buffer[2]

[{'translation_text': 'One of the most beautiful national parks in Chile'},
 {'translation_text': 'In this video you will be able to appreciate the works of the artist Alexander Cruz. All images have copyright ©.'},
 {'translation_text': 'We visit the fabulous finca Las Alberquillas with our friends of Dehesas&Umbrias.'}]

In [ ]:
translator(text, src_lang=source_language, tgt_lang=target_language, max_length=400)

In [ ]:
! pip freeze > requirements.txt